### Understanding a Decision Tree

A decision tree is the building block of a random forest and is an intuitive model. 

We can think of a decision tree as a series of yes/no questions asked about our data eventually leading to a predicted class (or continuous value in the case of regression). This is an interpretable model because it makes classifications much like we do: we ask a sequence of queries about the available data we have until we arrive at a decision (in an ideal world).

The technical details of a decision tree are in how the questions about the data are formed. 

**In the CART algorithm, a decision tree is built by determining the questions (called splits of nodes) that, when answered, lead to the greatest reduction in Gini Impurity. What this means is the decision tree tries to form nodes containing a high proportion of samples (data points) from a single class by finding values in the features that cleanly divide the data into classes.**

<p>
    <img src = assets/1.png height = 400px width = 400px>
</p>

Our data only has two features (predictor variables), x1 and x2 with 6 data points — samples — divided into 2 different labels. Although this problem is simple, it’s not linearly separable, which means that we can’t draw a single straight line through the data to classify the points.


We can however draw a series of straight lines that divide the data points into boxes, which we’ll call nodes. In fact, this is what a decision tree does during training. Effectively, a decision tree is a non-linear model built by constructing many linear boundaries.

#### Visualizing a Decision Tree

So what’s actually going on when we train a decision tree?

<p>
    <img src = assets/2.png height = 600px width = 600px>
</p>

All the nodes, except the leaf nodes (colored terminal nodes), have 5 parts:

- Question asked about the data based on a value of a feature. Each question has either a True or False answer that splits the node. Based on the answer to the question, a data point moves down the tree.
- `gini`: The Gini Impurity of the node. The average weighted Gini Impurity decreases as we move down the tree.
- `samples`: The number of observations in the node.
- `value`: The number of samples in each class. For example, the top node has 2 samples in class 0 and 4 samples in class 1.
- `class`: The majority classification for points in the node. In the case of leaf nodes, this is the prediction for all samples in the node.

**The leaf nodes do not have a question because these are where the final predictions are made. To classify a new point, simply move down the tree, using the features of the point to answer the questions until you arrive at a leaf node where the `class` is the prediction.**

To make see the tree in a different way, we can draw the splits built by the decision tree on the original data.

<p>
    <img src = assets/3.png height = 400px width = 400px>
</p>

Each split is a single line that divides data points into nodes based on feature values. For this simple problem and with no limit on the maximum depth, the divisions place each point in a node with only points of the same class. (Again, later we’ll see that this perfect division of the training data might not be what we want because it can lead to overfitting.)

### Gini Impurity:

**The Gini Impurity of a node is the probability that a randomly chosen sample in a node would be incorrectly labeled if it was labeled by the distribution of samples in the node.** For example, in the top (root) node, there is a 44.4% chance of incorrectly classifying a data point chosen at random based on the sample labels in the node. We arrive at this value using the following equation:

<p>
    <img src = assets/4.png height = 400px width = 400px>
</p>


The Gini Impurity of a node n is 1 minus the sum over all the classes J (for a binary classification task this is 2) of the fraction of examples in each class p_i squared. That might be a little confusing in words, so let’s work out the Gini impurity of the root node.

<p>
    <img src = assets/5.png>
</p>

**At each node, the decision tree searches through the features for the value to split on that results in the greatest reduction in Gini Impurity.**

It then repeats this splitting process in a greedy, recursive procedure until it reaches a maximum depth, or each node contains only samples from one class. The weighted total Gini Impurity at each level of tree must decrease. At the second level of the tree, the total weighted Gini Impurity is 0.333:

<p>
    <img src = assets/6.png>
</p>

**(The Gini Impurity of each node is weighted by the fraction of points from the parent node in that node.)**

Eventually, the weighted total Gini Impurity of the last layer goes to 0 meaning each node is completely pure and there is no chance that a point randomly selected from that node would be misclassified. While this may seem like a positive, it means that the model may potentially be overfitting because the nodes are constructed only using training data.

### Overfitting: Or Why a Forest is better than One Tree:

You might be tempted to ask why not just use one decision tree? It seems like the perfect classifier since it did not make any mistakes! A critical point to remember is that the tree made no mistakes on the training data. We expect this to be the case since we gave the tree the answers and didn’t limit the max depth (number of levels). The objective of a machine learning model is to generalize well to new data it has never seen before.

**Overfitting occurs when we have a very flexible model (the model has a [high capacity](https://www.quora.com/What-are-flexible-statistical-learning-methods/answer/Chekkal-Samir?share=fc6242eb&srid=h2tHX)) which essentially memorizes the training data by fitting it closely. The problem is that the model learns not only the actual relationships in the training data, but also any noise that is present. A flexible model is said to have high variance because the learned parameters (such as the structure of the decision tree) will vary considerably with the training data.
On the other hand, an inflexible model is said to have high bias because it makes assumptions about the training data (it’s biased towards pre-conceived ideas of the data.) For example, a linear classifier makes the assumption that the data is linear and does not have the flexibility to fit non-linear relationships. An inflexible model may not have the capacity to fit even the training data and in both cases — high variance and high bias — the model is not able to generalize well to new data.**


`The balance between creating a model that is so flexible it memorizes the training data versus an inflexible model that can’t learn the training data is known as the bias-variance tradeoff and is a foundational concept in machine learning.`


**The reason the decision tree is prone to overfitting when we don’t limit the maximum depth is because it has unlimited flexibility, meaning that it can keep growing until it has exactly one leaf node for every single observation, perfectly classifying all of them. If you go back to the image of the decision tree and limit the maximum depth to 2 (making only a single split), the classifications are no longer 100% correct. We have reduced the variance of the decision tree but at the cost of increasing the bias.**

**As an alternative to limiting the depth of the tree, which reduces variance (good) and increases bias (bad), we can combine many decision trees into a single ensemble model known as the random forest.**

## Random Forest

The random forest is a model made up of many decision trees. Rather than just simply averaging the prediction of trees (which we could call a “forest”), this model uses two key concepts that gives it the name random:

- Random sampling of training data points when building trees

When training, each tree in a random forest learns from a random sample of the data points. The samples are drawn with replacement, known as bootstrapping, which means that some samples will be used multiple times in a single tree. The idea is that by training each tree on different samples, although each tree might have high variance with respect to a particular set of the training data, overall, the entire forest will have lower variance but not at the cost of increasing the bias.

At test time, predictions are made by averaging the predictions of each decision tree. This procedure of training each individual learner on different bootstrapped subsets of the data and then averaging the predictions is known as bagging, short for [bootstrap aggregating](https://machinelearningmastery.com/bagging-and-random-forest-ensemble-algorithms-for-machine-learning/).

- Random subsets of features considered when splitting nodes

The other main concept in the random forest is that only a [subset of all the features are considered for splitting each node in each decision tree](https://sebastianraschka.com/faq/docs/random-forest-feature-subsets.html). Generally this is set to sqrt(n_features) for classification meaning that if there are 16 features, at each node in each tree, only 4 random features will be considered for splitting the node. (The random forest can also be trained considering all the features at every node as is common in regression. 

### When to use Gini Impurity and when to use Entropy?

**They are pretty much the same when it comes to CART Analytics.**

Generally, your performance will not change whether you use Gini impurity or Entropy. 

Laura Elena Raileanu and Kilian Stoffel compared both in "[Theoretical comparison between the gini index and information gain criteria](https://www.unine.ch/files/live/sites/imi/files/shared/documents/papers/Gini_index_fulltext.pdf)". The most important remarks were:

- It only matters in 2% of the cases whether you use gini impurity or entropy. 
- Entropy might be a little slower to compute (because it makes use of the logarithm). 

I was once told that **both metrics exist because they emerged in different disciplines of science.**

Also, 
- Gini is intended for continuous attributes and Entropy is for attributes that occur in classes
- Gini is to minimize misclassification while Entropy is for exploratory analysis

### [Intuition behind Gini impurity formula](https://stats.stackexchange.com/a/308886/210548)
### [Gini Impurity derivation](https://stats.stackexchange.com/a/339514/210548)

### REFERENCES:
- [Random Forest Implementation and Explanation](https://towardsdatascience.com/an-implementation-and-explanation-of-the-random-forest-in-python-77bf308a9b76)